# Time-coupled instanton: from one time step to many

The multi-time-step instanton problem has the following components:

1. Quadratic objective function of deviation variables
2. Linear power balance constraints
3. Quadratic line temperature constraint

Here I describe each component separately. I also include code that may be used to generate each component.
___

## 1. Objective Function

The objective function weights only the deviation variables $x$. For two time steps, we have:

\begin{align}
\min & \begin{bmatrix} x^{(1)\top} & \theta^{(1)\top} & \alpha^{(1)} & x^{(2)\top} & \theta^{(2)\top} & \alpha^{(2)} \end{bmatrix}
\underbrace{
\begin{bmatrix} Q_x & 0 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 & 0 \\ 0 & 0  & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & Q_x & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 & 0\end{bmatrix}}_{Q_{obj}}
\begin{bmatrix} x^{(1)} \\ \theta^{(1)} \\ \alpha^{(1)} \\ x^{(2)} \\ \theta^{(2)} \\ \alpha^{(2)} \end{bmatrix} \\
&= \sum_{t=1}^{T} x^{(t)\top}Q_xx^{(t)}
\end{align}

### Computing the objective function

The following method generates the matrix $Q_{obj}\in\mathbb{R}^{2nT\times 2nT}$, where each $Q_x$ is the identity matrix (no correlation):

In [ ]:
function tmp_inst_Qobj(n,nr,T)
    """ Generate the objective function matrix
    Qobj from the problem dimensions.
    Assume no correlation between wind sites.
    """
    return sparse(diagm(repeat([ones(nr),zeros(n)],outer=[T])))
end

## 2. Power Balance Constraints

Suppose our network has three nodes. Nodes 1 and 2 have wind farms, and node 3 is the angle reference. After removing the third row and column, our admittance matrix is

$$ Y = \begin{bmatrix} y_{11} & y_{12} \\ y_{12} & y_{22} \end{bmatrix}. $$

Assuming there are two time steps (just enough to see the pattern), the vector of variables is

\begin{align}
z &= \begin{bmatrix} z^{(1)} \\ z^{(2)} \end{bmatrix} \text{, where} \\
z^{(1)} &= \begin{bmatrix} x_1^{(1)} & x_2^{(1)} & x_3^{(1)} & \theta_1^{(1)} & \theta_2^{(1)} & \alpha^{(1)} \end{bmatrix}^\top \\
z^{(2)} &= \begin{bmatrix} x_1^{(2)} & x_2^{(2)} & x_3^{(2)} & \theta_1^{(2)} & \theta_2^{(2)} & \alpha^{(2)} \end{bmatrix}^\top
\end{align}
There are four linear equations per time step describing power balance and distributed slack. Let's consider the four equations of the first time step. The first describes $\alpha$, the mismatch variable necessary to implement distributed slack. 

\begin{align}
\alpha^{(1)} &= 1^\top(D^{(1)} - x^{(1)} - G^{0(1)}) \\
&= (D_1^{(1)} + D_2^{(1)} + D_3^{(1)}) - (x_1^{(1)} + x_2^{(1)} + x_3^{(1)}) - (G_1^{0(1)} + G_2^{0(1)} + G_3^{0(1)})
\end{align}
Distributed slack uses a vector of generator participation factors to assign a pre-defined portion of the power mistmatch to each generator. We can express the actual generation at a time step as a function of the dispatch values $G^0$ and the mismatch $\alpha$:

\begin{equation}
G_i^{(1)} = G_i^{0(1)} + \alpha^{(1)} k_i
\end{equation}
Intuitively, this means that when demand exceeds total generation, $\alpha$ will be positive, and each generator $G_i$ will increase its output by a fraction $k_i$ of $\alpha$, thereby balancing the mismatch.

Having described the behavior of conventional generators, we are ready to relate the bus angles to power injections at nodes 1 and 2:

\begin{align}
y_{11}\theta_1^{(1)} + y_{12}\theta_2^{(1)} - x_1^{(1)} &= G_1^{(1)} - D_1^{(1)} \\
&= (G_1^{0(1)} + \alpha^{(1)} k_1) - D_1^{(1)} \\
\implies \begin{bmatrix} -1 & 0 & 0 & y_{11} & y_{12} & -k_1 \end{bmatrix} z^{(1)} &= G_1^{0(1)} - D_1^{(1)} \\
\\
y_{12}\theta_1^{(1)} + y_{22}\theta_2^{(1)} - x_2^{(1)} &= G_2^{(1)} - D_2^{(1)} \\
&= (G_2^{0(1)} + \alpha^{(1)} k_2) - D_2^{(1)} \\
\implies \begin{bmatrix} 0 & -1 & 0 & y_{12} & y_{22} & -k_2 \end{bmatrix} z^{(1)} &= G_2^{0(1)} - D_2^{(1)} \\
\end{align}
At node 3 we know the angle is zero, but we still must express the relationship between wind deviation and conventional generation:

\begin{align}
-x_3^{(1)} - \alpha^{(1)}k_3 &= G_3^{0(1)} - D_3^{(1)} \\
\implies \begin{bmatrix} 0 & 0 & -1 & 0 & 0 & -k_3 \end{bmatrix} z^{(1)} &= G_3^{0(1)} - D_3^{(1)}
\end{align}
We can rewrite the equation for $\alpha$ in inner product form:

\begin{align}
\begin{bmatrix} 1 & 1 & 1 & 0 & 0 & 1 \end{bmatrix} z^{(1)} &= \sum_{i=1}^3 D_i^{(1)} -G_i^{0(1)}
\end{align}

___

Now let's put everything together. The four power balance equations may be written as a single linear equation in $z^{(1)}$:

\begin{align}
\begin{bmatrix}
-1 & 0 & 0 & y_{11} & y_{12} & -k_1 \\
0 & -1 & 0 & y_{12} & y_{22} & -k_2 \\
0 & 0 & -1 & 0 & 0 & -k_3 \\
1 & 1 & 1 & 0 & 0 & 1 \\
\end{bmatrix}
\begin{bmatrix} x_1^{(1)} \\ x_2^{(1)} \\ x_3^{(1)} \\ \theta_1^{(1)} \\ \theta_2^{(1)} \\ \alpha^{(1)} \end{bmatrix} &=
\begin{bmatrix}
G_1^{0(1)} - D_1^{(1)} \\
G_2^{0(1)} - D_2^{(1)} \\
G_3^{0(1)} - D_3^{(1)} \\
\sum_{i=1}^3 D_i^{(1)} -G_i^{0(1)}
\end{bmatrix}
\end{align}
Generalizing, we find that 

* There are $(n+n_r)T$ variables --where $n$ is the number of nodes, $n_r$ is the number of nodes with wind farms, and $T$ is the number of time steps
    * $n_rT$ of these are deviation variables,
    * $(n-1)T$ are angle variables, and 
    * $T$ are mismatch variables.

* There are $(n+1)T$  equations, where 
    * $nT$ describe injections and 
    * $T$ describe mismatches.

* Thus, the power balance equations are of the form $Az = b$, where
    * $A\in\mathbb{R}^{(n+1)T\times (n+n_r)T}$
    * $z\in\mathbb{R}^{(n+n_r)T\times 1}$
    * $b\in\mathbb{R}^{(n+1)T\times 1}$

### Computing power balance constraint parameters

The following methods generate $A$ and $b$:

In [ ]:
function tmp_inst_A(n,Ridx,T,Y,k)
    """ Generate the power balance constraint A matrix
    from problem dimensions, admittance matrix,
    and generator participation factors.
    Assumes the admittance matrix is (n-1)-by-(n-1).
    """
    # A = zeros((n+1)*T,2*n*T)
    
    # A has a block diagonal pattern where each
    # block is Atemp:
    Atemp = zeros((n+1),2*n)
    Atemp[1:n,1:n] = -eye(n)
    Atemp[end,1:n] = ones(1,n)
    Atemp[1:n-1,n+1:2n-1] = Y
    Atemp[:,2*n] = [-k,1]
    
    # Remove columns corresponding to non-wind nodes:
    Atemp = sparse(Atemp[:,[Ridx,n+1:2*n]])
    
    # Now we can tile the Atemp matrix to generate A:
    A = Atemp
    for t = 2:T
        A = blkdiag(A, Atemp)
    end
    
    return A
end

function tmp_inst_b(n,T,G0,D)
    """ Generate the vector b of power balance constraints.
    Assumes G0 and D are nT-by-1 vectors.
    """
    b = FloatingPoint[]
    
    # b = zeros((n+1)*T,1)
    netGen = G0 - D
    for t = 1:T
        start = (t-1)*n + 1
        stop = start + n - 1
        mismatch = -sum(netGen[start:stop])
        append!(b,netGen[start:stop])
        push!(b,mismatch)
    end
    return sparse(b)
end

## 3. Line Temperature Constraint

The purpose of our optimization routine is to find wind patterns that excessively heat a line of our choosing. This is expressed mathematically in terms of the angle variables:

\begin{align}
\begin{bmatrix} x^{(1)\top} & \theta^{(1)\top} & \alpha^{(1)} & x^{(2)\top} & \theta^{(2)\top} & \alpha^{(2)} \end{bmatrix}
\underbrace{\begin{bmatrix}
0 & Q_\theta^{(1)} & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0  & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & Q_\theta^{(2)} & 0 \\
0 & 0 & 0 & 0 & 0 & 0\end{bmatrix}}_{Q_\theta}
\begin{bmatrix} x^{(1)} \\ \theta^{(1)} \\ \alpha^{(1)} \\ x^{(2)} \\ \theta^{(2)} \\ \alpha^{(2)} \end{bmatrix} &= c \text{, where} \\
Q_\theta^{(t)} &= \tau^{T -t} (e_i - e_k)(e_i - e_k)^\top
\end{align}

### Computing line temp constraint parameters

The following method computes $Q_\theta$.

In [ ]:
function tmp_inst_Qtheta(n,nr,T,tau,line)
    """ Generate Q_theta in the temperature constraint
    of a temporal instanton problem instance.
    "line" has the form (i,k), where i and k refer to
    the endpoints of the chosen line.
    """
    Qtheta = zeros((n+nr)*T,(n+nr)*T)
    i,k = line
    ei = zeros(n-1,1)
    ei[i] = 1
    ek = zeros(n-1,1)
    ek[k] = 1
    
    Q0 = (ei - ek)*(ei - ek)'
    
    for t = 1:T
        start = nr + 1 + (nr+n)*(t-1)
        stop = start + n - 2
        Qtheta[start:stop,start:stop] = tau^(T-t)*Q0
    end
    return sparse(Qtheta)
end

___

# Problem Instance

The preceding discussion has considered each component separately. A complete instance of the temporal instanton problem is characterized by putting the pieces together:

* $n$, $n_r$, and $T$

* $Q_x$, which depends on desired correlation

* $A$, which depends on admittance matrix $Y$

* $b$, which depends on $G^0$ and $D$

* $Q_\theta$, which depends on $\tau$ and the line we wish to overload

Let's make up some numbers and generate a problem instance to ensure everything works.

In [ ]:
n = 4 # nodes
T = 3 # time steps

tau = 2 # heating coeff.

line = (1,3) # line to overheat

Y = [1 -2 -3; 
    -2 1 -4; 
    -3 -4 1] # admittance matrix

k = [0,0.25,0.25,0.5] # gen. participation factors

G0 = [0, 0.6, 0.3, 0.5]
G0 = [G0,0.75*G0,1.2*G0]
D = [0.5,0.2,0.2,0.4]
D = [D,0.7*D,1.2*D]

function temporalInstanton(n,t,tau,Y,k,G0,D,line)
    Qobj = generateQobj(n,T)
    A = generateA(n,T,Y,k)
    b = generateb(n,T,G0,D)
    Qtheta = generateQtheta(n,T,tau,line)
    return Qobj, A, b, Qtheta
end